In [1]:
! pip install -U accelerate
! pip install -U transformers

In [2]:

#!pip install transformers datasets
!pip install accelerate==1.1.1

In [3]:
#!pip install --upgrade pip
!pip install torch
#!pip install --upgrade numpy
#!pip install --upgrade pandas pandasai
#!pip uninstall numpy --yes
#!pip install datasets
#!pip install torch==2.0.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

In [4]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load a pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Ensure model runs on GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [6]:
from datasets import load_dataset

# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use a subset for testing


In [ ]:

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["article"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    # Tokenize summaries as target labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    #eval_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/var/folders/fn/5zh9ry610g9dwdm4l7ch98g80000gn/T/ipykernel_59028/2924761325.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/2154 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`article` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [11]:
def summarize(text, max_length=50, min_length=10, length_penalty=2.0, num_beams=4):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
text = """
Wann der Verkehr wieder nach Plan läuft, sei noch unklar, sagte ein Bahn-Sprecher am Mittwochmorgen. "Die Störung wird sich voraussichtlich bis zum späten Nachmittag hinziehen", ergänzte eine Sprecherin am Mittag.

Welche Fernverkehrsverbindungen sind betroffen?
alle ICE-Züge zwischen Hamburg und München, über Berlin, Halle(S)/Leipzig, Erfurt und Nürnberg fallen zwischen Berlin und Hamburg in beiden Richtungen aus
alle ICE-Züge zwischen Kiel/Lübeck/Hamburg und München, über Hannover, Göttingen, Kassel, Würzburg und Nürnberg fallen zwischen Hamburg und Hannover in beiden Richtungen aus
alle ICE/IC-Züge zwischen Stralsund und Karlsruhe, über Hamburg, Hannover, Kassel und Frankfurt fallen zwischen Hamburg und Karlsruhe aus
alle ICE-Züge zwischen Kiel/Hamburg und München über Hannover, Frankfurt und Stuttgart werden umgeleitet und verspäten sich um etwa 20 Minuten
alle ICE-Züge zwischen Kiel/Hamburg und Interlaken Ost, über Hannover, Frankfurt, Mannheim, Stuttgart und Basel SBB werden umgeleitet und verspäten sich um etwa 20 Minuten
alle ICE/IC-Züge zwischen Hamburg und Frankfurt(M)/Süddeutschland, über Hannover, Fulda und Frankfurt(M) werden umgeleitet und verspäten sich um etwa 20 Minuten. Züge halten nicht in Lüneburg, Bad Bevensen, Uelzen, Celle und Langenhagen
Reisende nach Hannover sollten ab Hamburg die Fernverkehrs- und Regionalzüge bis Bremen nutzen und dort umsteigen.
Für Reisende nach Frankfurt am Main wird empfohlen, die Direktverbindungen des Fernverkehrs ab Hamburg Hauptbahnhof zu nutzen.
NDR-Reporterin Marlene Kukral berichtet live von einem Bahnsteig in Lüneburg. © Screenshot 3 Min
Durch Blitzeinschlag: Bahnverkehr in Niedersachsen gestört
In der Nacht strandeten mehrere Züge und Passagiere mussten evakuiert werden. Marlene Kukral berichtet live aus Lüneburg. (27.11.2024)

Regionalverkehr der Linie RE3/RB31 läuft wieder
Einschränkungen gab es auch im Regionalverkehr. Auf der Linie RE3/RB31 wurde die Streckensperrung zwischen Lüneburg und Uelzen am Mittwochnachmittag in beide Richtungen wieder aufgehoben, wie das private Bahnunternehmen Metronom mitteilte. Es komme allerdings weiter zu Teilausfällen und Verspätungen durch den Rückstau wartender Züge. Der Busnotverkehr sei wieder eingestellt worden.

Vor Antritt der Fahrt unbedingt informieren
Metronom und Deutsche Bahn raten Reisenden, sich vor dem Antritt ihrer Fahrt über ihre Verbindung zu informieren. Tickets des heutigen Tages könnten zu einem späteren Zeitpunkt genutzt werden, da die Zugbindung aufgehoben wurde. Sitzplatzreservierungen könnten außerdem kostenfrei storniert werden.

ICE-Passagiere mussten in Busse umsteigen
Am Dienstagabend war ein Blitz in einen Baum an der Bahnstrecke in der Gemeinde Deutsch Evern (Landkreis Lüneburg) eingeschlagen. Ein Teil des Baumes fiel laut Deutscher Bahn auf eine Oberleitung. Ein ICE aus München sei in der Folge kurz hinter dem Bahnhof Bienenbüttel zum Stehen gekommen, teilte die Gemeindefeuerwehr mit. Die etwa 350 Fahrgäste konnten den Zug auf freier Strecke nicht verlassen. Eine Diesellok aus Hannover schleppte den ICE schließlich nach etwa viereinhalb Stunden in den Bahnhof. Dort wurden die Reisenden laut der Mitteilung vom Deutschen Roten Kreuz unter anderem mit Getränken und Decken versorgt, bevor es mit Bussen weiter nach Hamburg ging.
"""
print(summarize(text))


Wann der Verkehr wieder nach Plan läuft, sei noch unklar, sagte ein Bahn-Sprecher. "Die Störung wird sich vora


In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(references, skip_special_tokens=True)
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure for key, value in result.items()}
    return result
